# ~~broken~~ SCACO for TSP 
Made by [Kishkenebayeva Altynay](https://github.com/Cupcakeiris) & Shamitova Lazzat for Optimization classes | Fall 2024

Using this cool [paper](https://www.researchgate.net/publication/228617274_Improved_ant_colony_optimization_algorithm_for_the_traveling_salesman_problems) for the realization

### Initialization

Libraries

In [220]:
import numpy as np
import pandas as pd
from numba import njit
import matplotlib.pyplot as plt
import time
from scipy.stats import zscore
import os

In [221]:
real_solution = 9352
start_time = time.time()

Hyperparameters

In [222]:
alpha = 2.0                 # Importance of pheromone
beta = 4.0                  # Importance of visibility (1/distance)
evaporation_rate = 0.95     # Pheromone evaporation rate
num_ants = 10
scout_rate = 0.2            # Fraction of ants that are scouts
num_scouts = int(num_ants * scout_rate)
num_iterations = 3000
real_solution = 9352

In [223]:
df = pd.read_csv('qa194.csv', sep=' ')
df.describe()

,x,y
count,194.000000,194.000000
mean,25555.658645,51292.371134
std,330.297453,201.368050
min,24748.333300,50766.666700
25%,25283.888875,51159.236100
50%,25493.611100,51322.361150
75%,25855.000025,51457.430575
max,26150.277800,51619.166700


In [224]:
# To work with "dimensionless" data to find the best route. Then find total distance of this route
df[['x', 'y']] = df[['x', 'y']].apply(zscore)
coordinates = df[['x', 'y']].to_numpy() 

### The hell

I/O for last best path

In [225]:
def read_best_path(filename='best_path.txt'):
    if os.path.exists(filename):
        return np.loadtxt(filename, dtype=np.int32)
    else:
        return None 
    
def save_best_path(best_path, filename='best_path.txt'):
    np.savetxt(filename, best_path, fmt='%d')

In [226]:
%%latex
\begin{align}
    p_{ir}^k = \begin{cases}
        \frac{\tau_{ir}^\alpha(t) \eta_{ir}^\beta(t)}{\sum_{l \neq tabu} \tau_{il}^\alpha(t) \eta_{il}^\beta(t)} \cdot \left( (r \notin tabu) \cap (p < M_{ir}) \cap (d_{ir} \leq d_{ij}) \right) \\
        0, (r \in tabu) \cup (r \notin tabu \cap (p \geq M_{ir}))
    \end{cases} \tag{9}
\end{align}

But ignore $$M_{ir}$$ since we don't apply it in this lab

<IPython.core.display.Latex object>

In [227]:
@njit
def calculate_transition_probabilities(pheromone_matrix, dist_matrix, current_city, visited):
    num_cities = len(pheromone_matrix)
    pheromone = pheromone_matrix[current_city]
    visibility = 1 / dist_matrix[current_city]

    probabilities = np.zeros(num_cities)
    for next_city in range(num_cities):
        if visited[next_city] == 0:  # City is not visited
            probabilities[next_city] = (pheromone[next_city] ** alpha) * (visibility[next_city] ** beta)

    total = probabilities.sum()
    if total > 0:
        probabilities /= total  
    else:
        probabilities.fill(0)

    return probabilities

In [228]:
%%latex
\begin{align}
    \tau_{ij}(t+m) &= \rho \tau_{ij}(t) + \Delta \tau_{ij}, \quad \rho \in (0,1) \tag{2} \\
    \Delta \tau_{ij} &= \sum_{k=1}^n \Delta \tau_{ij}^k \tag{3} \\
    \Delta \tau_{ij}^k &= \begin{cases}
        \frac{Q}{L^k}, & \text{arc }(i,j) \text{ belongs to best tour} \\
        0, & \text{otherwise}
    \end{cases} \tag{4}
\end{align}

<IPython.core.display.Latex object>

In [229]:
@njit
def update_pheromone(pheromone_matrix, all_paths, all_distances, num_cities):
    pheromone_matrix *= (1 - evaporation_rate)  # Evaporation
    for ant in range(len(all_paths)):
        path = all_paths[ant]
        distance = all_distances[ant]
        pheromone_addition = 1 / distance
        for i in range(num_cities):
            pheromone_matrix[path[i], path[i + 1]] += pheromone_addition
            pheromone_matrix[path[i + 1], path[i]] += pheromone_addition

In [230]:
%%latex
For search ants{
    Randomly choose the start of city;
    While (NOT all city visited){
        Calculate the following city according to (2);
    }
}

<IPython.core.display.Latex object>

In [231]:
@njit
def construct_solution(pheromone_matrix, dist_matrix):
    num_cities = len(dist_matrix)
    start_city = np.random.randint(num_cities)
    path = np.empty(num_cities + 1, dtype=np.int32)  
    path[0] = start_city
    visited = np.zeros(num_cities, dtype=np.int32)
    visited[start_city] = 1

    current_city = start_city
    for i in range(1, num_cities):
        probabilities = calculate_transition_probabilities(pheromone_matrix, dist_matrix, current_city, visited)
        
        # Select next city
        random_value = np.random.rand()
        cumulative_probability = 0.0
        next_city = -1
        for j in range(num_cities):
            cumulative_probability += probabilities[j]
            if random_value < cumulative_probability:
                next_city = j
                break

        path[i] = next_city
        visited[next_city] = 1
        current_city = next_city

    path[-1] = start_city  # Return to the start
    return path

In [232]:
%%latex
We gave up to implement formula (8), just decided to randomly shuffle the route for the mutation

<IPython.core.display.Latex object>

In [233]:
@njit
def construct_random_solution(num_cities):
    # Generate a random path for a scout ant (mutation)
    path = np.arange(num_cities, dtype=np.int32)  
    np.random.shuffle(path)
    path = np.append(path, path[0])  
    return path

In [234]:
@njit
def calculate_path_distance(path, dist_matrix):
    total_distance = 0.0
    for i in range(len(path) - 1):
        total_distance += dist_matrix[path[i], path[i + 1]]
    return total_distance

In [235]:
@njit
def ant_colony_optimization(dist_matrix):
    num_cities = len(dist_matrix)
    pheromone_matrix = np.ones((num_cities, num_cities), dtype=np.float64)

    best_path = np.empty(num_cities + 1, dtype=np.int32) 
    best_distance = np.inf

    all_paths = np.empty((num_ants, num_cities + 1), dtype=np.int32) 
    all_distances = np.empty(num_ants)

    for iteration in range(num_iterations):
        
        # Each ant constructs a solution
        for ant in range(num_ants):
            if ant < num_scouts:
                # Scout ant generates a random solution
                path = construct_random_solution(num_cities)
            else:
                # Regular ant follows pheromone trail
                path = construct_solution(pheromone_matrix, dist_matrix)
            
            distance = calculate_path_distance(path, dist_matrix)
            all_paths[ant] = path
            all_distances[ant] = distance

            if distance < best_distance:
                best_distance = distance
                best_path[:] = path.copy()   

        
        update_pheromone(pheromone_matrix, all_paths, all_distances, num_cities)
        # print(f"Iteration {iteration + 1}, best path distance: {best_distance}")

    return best_path, best_distance

In [236]:
@njit
def calculate_distance_matrix(coordinates):
    num_cities = len(coordinates)
    dist_matrix = np.zeros((num_cities, num_cities), dtype=np.float64) 
    for i in range(num_cities):
        for j in range(i + 1, num_cities):
            dist_matrix[i][j] = dist_matrix[j][i] = np.sqrt((coordinates[i][0] - coordinates[j][0]) ** 2 +
                                                            (coordinates[i][1] - coordinates[j][1]) ** 2)
    return dist_matrix

In [237]:
@njit
def calculate_distance_for_given_path(coordinates, path): # To return back the "dimension"
    total_distance = 0.0
    num_cities = len(path)
    
    for i in range(num_cities - 1):
        city1 = path[i]
        city2 = path[i + 1]
        
        dx = coordinates[city1, 0] - coordinates[city2, 0]
        dy = coordinates[city1, 1] - coordinates[city2, 1]
        distance = np.sqrt(dx * dx + dy * dy)
        
        total_distance += distance
    
    start_city = path[0]
    end_city = path[-1]
    dx = coordinates[start_city, 0] - coordinates[end_city, 0]
    dy = coordinates[start_city, 1] - coordinates[end_city, 1]
    total_distance += np.sqrt(dx * dx + dy * dy)
    
    return total_distance


### The result

Return back the "dimensions"

In [238]:
df = pd.read_csv('qa194.csv', sep=' ')
coordinates = df[['x', 'y']].to_numpy() 

In [239]:
initial_path = read_best_path()
if initial_path is not None:
    print("Using previous best path as initial solution.")
else:
    initial_path = construct_random_solution(len(coordinates))

dist_matrix = calculate_distance_matrix(coordinates)

best_path, _ = ant_colony_optimization(dist_matrix)
best_distance = calculate_distance_for_given_path(coordinates, best_path)

print("Best path:", best_path + 1)
print("Best path distance:", best_distance)
print("Error rate compared to real solution:", (best_distance - real_solution) / real_solution)
print("Time taken:", time.time() - start_time)

Using previous best path as initial solution.
Best path: [150 153 157 154 139 138 149 146 142 137 140 145 156 161 163 164 183 186
 187 190 194 182 176 169 172 179 173 174 175 181 177 184 189 191 192 188
 193 185 180 178 168 165 159 158 162 167 170 171 166 160 151 155 148 143
 136 131 129 135 133 124 123 128 120 121 117 116 115 112 110 107 108 105
 106 118 122 119 114 113 109 102 103  91  93  96  95  97  92  88  83  81
  79  77  84 100  21  18  24  26  17  14  11   7   4   2   3   5   9  10
  12  15  19  50  55  49  46  48  53  52  54  44  42  30  32  35  31  38
  41  43  40  34  39  47  51  58  56  61  67  66  73  68  70  64  57  45
  37  27  22  29  28  33  60  69  74  72  75  78  76  87  80  71  25  23
  13  16   8   6   1  20  65  63  36  59  62  82  89  90  94  99 101 104
 111  98  86  85 130 134 132 127 125 126 144 141 152 147 150]
Best path distance: 10312.19597821518
Error rate compared to real solution: 0.1026727949331887
Time taken: 10.80647349357605


### Some extra

The most precise result we got (same params, 200k iterations, 52 min, 6th attempt)

![record](record.png)

I've been using VSCode since 9th grade, it never has crashed in my life until this lab (c) Altynay 

![hell](crash.png)